In [1]:
from iotbx.map_manager import map_manager as MapManager
from mmtbx.model import manager as ModelManager
from iotbx.data_manager import DataManager
from iotbx.map_model_manager import map_model_manager as MapModelManager
from iotbx.pdb import common_residue_names_get_class
from libtbx import group_args
from mmtbx.maps.correlation import five_cc
import os
import pickle
import inspect

Could not import the compiled Python-sander interface. Make sure you have the Python development libraries installed and that you have sourced amber.sh or amber.csh

In [2]:
import sys
sys.path.append("../")
from ligand_map_model_validation import extract_ligand_models, ligands_map_model_cc

## Instantiate a DataManager object from data on disk

In [3]:
model_path = "../data/6f1u_4169/6f1u_4169.cif"
map_path = "../data/6f1u_4169/6f1u_4169.map"
group_args_entry = "../data/6f1u_4169/6f1u_4169.pkl"
with open(group_args_entry,"rb") as fh:
    entry = pickle.load(fh)


dm = DataManager()
dm.process_model_file(model_path)
dm.process_real_map_file(map_path)

map_manager = dm.get_real_map()
map_manager.set_resolution(entry.resolution)
map_model_manager = MapModelManager(map_manager=map_manager,model=dm.get_model())

# this is necessary for cryo-em models without crystal symmetry
if model.crystal_symmetry() is None:
    from cctbx.maptbx.box import shift_and_box_model
    model = shift_and_box_model(model)

NameError: name 'model' is not defined

In [179]:
from phenix.programs import setup_tutorial
inspect.getabsfile(setup_tutorial)

'/home/user/Software/phenix/modules/phenix/phenix/programs/setup_tutorial.py'

In [143]:
def extract_ligand_models(model,include_resnames=[],exclude_resnames=[]):
    
    
    exclude_classes = ["common_amino_acid",
             "modified_amino_acid",
             "common_rna_dna",
             "modified_rna_dna",
             "ccp4_mon_lib_rna_dna",
             "common_water"]

    default_exclude_resnames= ["UNK"]
       
    exclude_resnames = default_exclude_resnames+exclude_resnames
    
    extract_all = True if len(include_resnames) ==0 else False
       
    ligand_dict = {}
    sel_str_other = "not (water or nucleotide or protein)"  # "other_cnts" in model.composition()
    other_model = model.select(model.selection(sel_str_other))
    # I profiled the above method compared to looping over the entire original model,
    # and this was >10X faster. 
    
    
    for model in other_model.get_hierarchy().models():
        for chain in model.chains():
            for rg in chain.residue_groups():
                for resname in rg.unique_resnames():
                    if (common_residue_names_get_class(name=resname) not in exclude_classes
                       and resname not in exclude_resnames):
                        if resname in include_resnames or extract_all:
                            iselection = rg.atoms().extract_i_seq()
                            id_tuple = (model.id, chain.id, rg.resseq, resname)
                            assert(id_tuple not in ligand_dict.keys())
                            ligand_model = other_model.select(iselection)
                            ligand_dict[id_tuple] = ligand_model
        
    return ligand_dict

In [144]:
def ligands_cc(map_model_manager,output_directory=None):
    mmm = map_model_manager
    mm = mmm.map_manager()
    ligand_dict = extract_ligand_models(model)
    
    return_ligand_dict = {}
    ccmask_values = []

    for ligand_id, ligand_model in ligand_dict.items():
        
        model_id, chain_id, rg_resseq, resname = ligand_id
        

        map_model_manager = MapModelManager(map_manager=mm,
                                            model=ligand_model,
                                            ignore_symmetry_conflicts=True)
        mmmlig = map_model_manager.extract_all_maps_around_model()
        return_ligand_dict[ligand_id] = mmmlig
        
        if mmm.map_manager()._resolution is None:
            # log that calculating resolution is very slow
            pass
        
        ccmask = mmmlig.map_model_cc(resolution=mm.resolution()) 
        # This uses the five_cc object internally, and returns only CCmask
        
        ccmask_values.append(ccmask)
        if output_directory is not None:
            if not os.path.exists(output_directory):
                os.path.mkdir(output_directory)

                ligand_name = resname.strip()+"_"+chain_id.strip()
                ligand_map_file = os.path.join(output_directory,ligand_name+".map")
                ligand_model_file = os.path.join(output_directory,ligand_name+".cif")

                dm.write_model_file(mmmlig.model(),filename=ligand_model_file,extension="cif",overwrite=True)
                dm.write_real_map_file(mmmlig.map_manager(),filename=ligand_map_file,overwrite=True)

    return return_ligand_dict, ccmask_values
        

In [145]:
ligand_dict, ccmask_values = ligands_cc(map_model_manager)


In [150]:
dm.write_phil_file("../debug/dbg.phil")

In [152]:
print(inspect.getsource(dm.write_phil_file))

  def write_phil_file(self, phil_str, filename=Auto, overwrite=Auto):
    # use this instead of libtbx.phil.scope.show for consistent error messages
    if filename is Auto:
      filename = self.get_default_output_phil_filename()
    self._write_text(PhilDataManager.datatype, phil_str,
                     filename=filename, overwrite=overwrite)



In [176]:
master_phil_str = """

  input_files {
    include scope iotbx.map_model_manager.map_model_phil_str
    }

  crystal_info {
    resolution = None
      .type = float
      .help = Nominal resolution of map
      .short_caption = Resolution
    }
"""

In [177]:
from libtbx.phil import parse

In [178]:
phil_scope = parse(master_phil_str)

In [175]:
from __future__ import absolute_import, division, print_function
from phenix.program_template import ProgramTemplate
from libtbx import group_args

# =============================================================================

class Program(ProgramTemplate):

  description = """
  Demo program to calculate map-model correlation
  Usage: phenix.map_cc model.pdb map.mrc 3

  """

  # Define the data types that will be used
  datatypes = ['model', 'phil', 'real_map']

  # Input parameters

  # Note: include of map_model_phil_str allows specification of
  #   full_map, half_map, another half_map, and model and Program Template
  #   produces input_files.map_model_manager containing these

  master_phil_str = """

  input_files {
    include scope iotbx.map_model_manager.map_model_phil_str
    }

  crystal_info {
    resolution = None
      .type = float
      .help = Nominal resolution of map
      .short_caption = Resolution
    }

  """

  # Define how to determine if inputs are ok
  def validate(self):

    # Expect exactly one map and one model. Stop if not the case
    self.data_manager.has_real_maps(
      raise_sorry = True,
      expected_n  = 1,
      exact_count = True)
    self.data_manager.has_models(
      raise_sorry = True,
      expected_n  = 1,
      exact_count = True)

  # Set any defaults
  def set_defaults(self):
    pass


  # Run the program
  def run(self):

    from phenix.model_building.map_cc import run as get_map_cc
    self.result = get_map_cc(
      map_model_manager = self.data_manager.get_map_model_manager(
         from_phil=True),
      resolution = self.params.crystal_info.resolution,
      log        = self.logger)

  # Get the results
  def get_results(self):
    return group_args(
      map_model_cc = self.result.map_model_cc,
     )

AttributeError: 'scope' object has no attribute 'crystal_info'

In [174]:
phil_scope.resolve_variables()

In [47]:
output_directory = "../data/6f1u_4169/ligands/"

NameError: name 'mmm' is not defined

In [13]:
mm

Map manager (from ../data/6f1u_4169/6f1u_4169.map)
Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1), 
Unit-cell grid: (600, 600, 600), (present: (600, 600, 600)), origin shift (0, 0, 0) 
Working coordinate shift (-0.0, -0.0, -0.0)

# Do a second ligand

In [15]:
map_model_manager_1 = MapModelManager(map_manager=mm,
                                    model=ligand_models[1],
                                    ignore_symmetry_conflicts=True)

In [16]:
boxed_mmm_1 = map_model_manager_1.extract_all_maps_around_model()
ligand_mm_1 = boxed_mmm_1.map_manager()
ligand_model_1 = boxed_mmm_1.model()


In [17]:
boxed_mmm_1


Map_model_manager: 

Model manager 
Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1)
Chains: 1 Residues 1 
Working coordinate shift (-435.49999999999994, -392.62000000000006, -410.04))

map_manager: Map manager (from ../data/6f1u_4169/6f1u_4169.map)
Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1), 
Unit-cell grid: (600, 600, 600), (present: (18, 15, 15)), origin shift [325, 293, 306] 
Working coordinate shift (-435.49999999999994, -392.62000000000006, -410.04)

In [18]:
boxed_mmm


Map_model_manager: 

Model manager 
Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1)
Chains: 1 Residues 1 
Working coordinate shift (-397.98, -397.9800000000001, -450.24000000000007))

map_manager: Map manager (from ../data/6f1u_4169/6f1u_4169.map)
Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1), 
Unit-cell grid: (600, 600, 600), (present: (18, 18, 13)), origin shift [297, 297, 336] 
Working coordinate shift (-397.98, -397.9800000000001, -450.24000000000007)

In [19]:
boxed_mmm_1.write_map("../data/6f1u_4169/ligand1.mrc")
boxed_mmm_1.write_model("../data/6f1u_4169/ligand1.pdb")